# Mueller module

In [2]:
%run ../config_jupyter.ipynb
%autoreload 2

from math import sqrt

from py_pol.mueller import Mueller
from py_pol.stokes import Stokes
from py_pol.jones_matrix import Jones_matrix
from py_pol.utils import rotation_matrix_Mueller, isrow, iscolumn, azimuth_elipt_2_charac_angles, list_of_objects

from py_pol import degrees
import numpy as np

eps = 1e-6

{'commit_hash': '523ed2fe5',
 'commit_source': 'installation',
 'default_encoding': 'cp1252',
 'ipython_path': 'C:\\Users\\Jesus\\Anaconda3\\lib\\site-packages\\IPython',
 'ipython_version': '7.2.0',
 'os_name': 'nt',
 'platform': 'Windows-10-10.0.17134-SP0',
 'sys_executable': 'C:\\Users\\Jesus\\Anaconda3\\python.exe',
 'sys_platform': 'win32',
 'sys_version': '3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit '
                '(AMD64)]'}
3.0.2


## Algebraic operations

### Rotation with keep = True

In [2]:
M1 = Mueller('M1')
M1.diattenuator_linear(p1=1, p2=0, angle=0*degrees)
print(M1)
print(M1.M)

c
M1 = 
  [+0.500, +0.500, +0.000, +0.000]
  [+0.500, +0.500, +0.000, +0.000]
  [+0.000, +0.000, +0.000, +0.000]
  [+0.000, +0.000, +0.000, +0.000]

[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]


In [3]:
M2=M1.rotate(angle=45*degrees, keep=True)
print(M1,'\n')
print(M2)

a
M1 @45.00º = 
  [+0.500, +0.000, +0.500, +0.000]
  [+0.000, +0.000, +0.000, +0.000]
  [+0.500, +0.000, +0.500, +0.000]
  [+0.000, +0.000, +0.000, +0.000]
 

None


### Rotation with keep = False

In [4]:
M1 = Mueller('M1')
M1.diattenuator_linear(p1=1, p2=0, angle=0*degrees)
print(M1)

c
M1 = 
  [+0.500, +0.500, +0.000, +0.000]
  [+0.500, +0.500, +0.000, +0.000]
  [+0.000, +0.000, +0.000, +0.000]
  [+0.000, +0.000, +0.000, +0.000]



In [5]:
M2=M1.rotate(angle=45*degrees, keep=False)
print(M1,'\n')
print(M2)

b
M1 = 
  [+0.500, +0.500, +0.000, +0.000]
  [+0.500, +0.500, +0.000, +0.000]
  [+0.000, +0.000, +0.000, +0.000]
  [+0.000, +0.000, +0.000, +0.000]
 

None


In [6]:
t=M1.vacuum()
print(t)

None


In [6]:
t=M1.get()
print(t)

[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]


### Multiplication of 2 Mueller Matrices

In [2]:
""" Test for matrix multiplication."""
solution = np.matrix(
    np.array([[96, 68, 69, 69], [24, 56, 18, 52], [58, 95, 71, 92],
              [90, 107, 81, 142]]))

M1 = np.matrix(
    np.array([[5, 2, 6, 1], [0, 6, 2, 0], [3, 8, 1, 4], [1, 8, 5, 6]]))
M2 = np.matrix(
    np.array([[7, 5, 8, 0], [1, 8, 2, 6], [9, 4, 3, 8], [5, 3, 7, 9]]))
Mueller1 = Mueller()
Mueller1.from_matrix(M1)
Mueller2 = Mueller()
Mueller2.from_matrix(M2)
Mueller3 = Mueller1 * Mueller2
proposal = Mueller3.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[ 96  68  69  69]
 [ 24  56  18  52]
 [ 58  95  71  92]
 [ 90 107  81 142]]
0.0


In [3]:
solution = np.matrix(
    np.array([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0],
              [0, 0, 0, 0]]))

M1 = Mueller('M1')
M1.diattenuator_linear(p1=1, p2=0, angle=0*degrees)
M2 = Mueller('M2')
M2.diattenuator_linear(p1=1, p2=0, angle=90*degrees)
M3 = M1 * M2
proposal = M3.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
0.0


In [9]:
solution = np.matrix([[1], [0], [0], [0]])
M1 = Mueller('M1')
M1.depolarizer([1, 0, 0])
J1 = Stokes('J1')
J1.from_elements(1, 0, 0, 1)
J2 = M1 * J1
proposal = J2.get()

print(proposal)
print(np.linalg.norm(proposal - solution))

[[1.]
 [0.]
 [0.]
 [0.]]
0.0


### Mueller matrix and stokes vector

In [4]:
# Matrix definition
solution = np.matrix(np.array([[0],[0],[0],[0]]))
M1 = Mueller('M1')
M1.diattenuator_linear(p1=1, p2=0, angle=0*degrees)
S1 = Stokes('S1')
S1.general_azimuth_ellipticity(azimuth=90*degrees, ellipticity=0, intensity=1)
S2 = M1*S1
proposal = S2.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[0.]
 [0.]
 [0.]
 [0.]]
0.0


### Numbers

In [4]:
# Division
M1 = Mueller('M1')
M1.diattenuator_linear(p1=1, p2=0, angle=0*degrees)
M2 = 5/M1
print(M2)

TypeError: unsupported operand type(s) for /: 'int' and 'Mueller'

## Matrix operations

In [5]:
# divide_in_blocks
solution1 = np.matrix(np.array([2, 3, 4]))
solution2 = np.matrix(np.array([[5],[9],[13]]))
solution3 = np.matrix(
    np.array([[6, 7, 8], [10, 11, 12],
              [14, 15, 16]]))
solution4 = 1
M = np.matrix(
    np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12],
              [13, 14, 15, 16]]))
M1 = Mueller('M1')
M1.from_elements(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)
(proposal1,proposal2,proposal3,proposal4) = M1.divide_in_blocks()

print(np.linalg.norm(proposal1 - solution1))
print(np.linalg.norm(proposal2 - solution2))
print(np.linalg.norm(proposal3 - solution3))
print(np.linalg.norm(proposal4 - solution4))


0.0
0.0
0.0
0.0


In [6]:
# Rotate
solution = np.matrix(
    np.array([[0.5, 0.5, 0, 0], [0.5, 0.5, 0, 0], [0, 0, 0, 0],
              [0, 0, 0, 0]]))
M = solution
M1 = Mueller('M1')
M1.from_matrix(M)
M1.rotate(0)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

solution = np.matrix(
    np.array([[0.5, 0, 0.5, 0], [0, 0, 0, 0], [0.5, 0, 0.5, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
M1.from_matrix(M)
M1.rotate(45*degrees)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

r3 = np.sqrt(3)
solution = np.matrix(
    np.array([[0.5, -0.25, r3/4, 0], [-0.25, 0.125, -r3/8, 0], [r3/4, -r3/8, 0.375, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
M1.from_matrix(M)
M1.rotate(60*degrees)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

solution = np.matrix(
    np.array([[0.5, -0.5, 0, 0], [-0.5, 0.5, 0, 0], [0, 0, 0, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
M1.from_matrix(M)
M1.rotate(90*degrees)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]
0.0
[[5.00000000e-01 3.06161700e-17 5.00000000e-01 0.00000000e+00]
 [3.06161700e-17 1.87469973e-33 3.06161700e-17 0.00000000e+00]
 [5.00000000e-01 3.06161700e-17 5.00000000e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
6.123233995736766e-17
[[ 0.5        -0.25        0.4330127   0.        ]
 [-0.25        0.125      -0.21650635  0.        ]
 [ 0.4330127  -0.21650635  0.375       0.        ]
 [ 0.          0.          0.          0.        ]]
2.2887833992611187e-16
[[ 5.00000000e-01 -5.00000000e-01  6.12323400e-17  0.00000000e+00]
 [-5.00000000e-01  5.00000000e-01 -6.12323400e-17  0.00000000e+00]
 [ 6.12323400e-17 -6.12323400e-17  7.49879891e-33  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
1.2246467991473532e-16


In [7]:
# Subtest: Rotation matrix mueller
solution = np.matrix(
    np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, -1, 0, 0],
              [0, 0, 0, 1]]))
proposal = rotation_matrix_Mueller(45*degrees)

print(proposal)
print(np.linalg.norm(proposal - solution))

[[ 1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  6.123234e-17  1.000000e+00  0.000000e+00]
 [ 0.000000e+00 -1.000000e+00  6.123234e-17  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
8.659560562354933e-17


## Definitions

### from_*

In [8]:
# from_elemnts
solution = np.matrix(
    np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12],
              [13, 14, 15, 16]]))
M1 = Mueller('M1')
M1.from_elements(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]
 [13. 14. 15. 16.]]
0.0


In [9]:
# from_matrix
solution = np.matrix(
    np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12],
              [13, 14, 15, 16]]))

M1 = Mueller('M1')
M1.from_matrix(solution)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
0.0


In [10]:
# from_Jones
solution = np.matrix(
    np.array([[0.5, 0.5, 0, 0], [0.5, 0.5, 0, 0], [0, 0, 0, 0],
              [0, 0, 0, 0]]))
JM = np.matrix(np.array([[1+0j, 0],[0, 0]], dtype=complex))
J1 = Jones_matrix('J')
J1.from_matrix(JM)
M1 = Mueller('M1')
M1.from_Jones(J1)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]
0.0


In [11]:
# from_blocks
solution = np.matrix(
    np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12],
              [13, 14, 15, 16]]))
D = np.matrix(np.array([2, 3, 4]))
P = np.matrix(np.array([[5],[9],[13]]))
m = np.matrix(
    np.array([[6, 7, 8], [10, 11, 12],
              [14, 15, 16]]))
m00 = 1

M1 = Mueller('M1')
M1.from_blocks(D, P, m, m00)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
0.0


In [10]:
# covariance_matrix and from_covariance
original = np.matrix(
    np.array([[1, -0.0229, 0.0027, 0.0058], [-0.0186, 0.9956, -0.0361, 0.0318], [-0.0129, 0.0392, 0.2207, -0.9656],
              [0.0014, 0.0280, 0.9706, 0.2231]]))
covariant = np.matrix(
    np.array([[0.4885, -0.0083+0.0094j, 0.0066-0.0073j, 0.1109-0.484j], [-0.0086-0.0094j, 0.0022, -0.0006-0.0013j, -0.013+0.0067j], 
              [0.0066+0.0073j, -0.0006+0.0013j, 0, 0.0097-0.0065j], [0.1109+0.484j, -0.013-0.0067j, 0.0097+0.0068j, 0.5093]]))
M1 = Mueller('M1')
M1.from_matrix(original)
proposal = M1.covariance_matrix()
print(np.linalg.norm(proposal - covariant))

M1.from_covariance(covariant)
proposal = M1.M
print(np.linalg.norm(proposal - original))






0.0004301162633521301
0.0008602325267042792


In [9]:
# from_inverse
N = 20
error= np.zeros(N)
for ind in range(N):
    (p1, p2, azimuth, ellipticity) = np.random.rand(4)
    original = Mueller()
    original.diattenuator_azimuth_ellipticity_from_vector(1.0/p1, 1.0/p2, azimuth, ellipticity)
    M1 = Mueller()
    M1.diattenuator_azimuth_ellipticity_from_vector(p1, p2, azimuth, ellipticity)
    proposal = M1.from_inverse(M1)
    error[ind] = np.linalg.norm(proposal - original.M)
print(error)

[2.48667876e-10 8.20167646e-15 9.00180757e-15 3.56020034e-16
 3.90257240e-16 5.61816067e-15 3.07273280e-15 9.22438715e-15
 2.21160057e-15 1.11636473e-15 1.12332627e-13 7.69768489e-16
 2.15066920e-11 7.41525929e-14 3.87640788e-13 1.10570354e-15
 1.17271508e-13 4.19840396e-16 2.34705850e-10 1.36247745e-14]


### Generators

**Diattenuators**

In [12]:
# diattenuator_linear

solution = np.matrix(
    np.array([[0.5, 0.5, 0, 0], [0.5, 0.5, 0, 0], [0, 0, 0, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
M1.diattenuator_linear(p1=1, p2=0, angle=0*degrees)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]
0.0


In [13]:
# diattenuator_linear_depolarizer
M1 = Mueller('M1')
try:
    M1.diattenuator_linear_depolarizer(p1=1, p2=0, d=0.5, angle=0*degrees, verbose=True)
    assert False
except:
    assert True

In [14]:
# diattenuator_charac_angles_from_Jones
solution = np.matrix(
    np.array([[0.5, 0.5, 0, 0], [0.5, 0.5, 0, 0], [0, 0, 0, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
M1.diattenuator_charac_angles_from_Jones(p1=1, p2=0, alpha=0, delay=0)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))


solution = np.matrix(
    np.array([[0.5, 0, -0.5, 0], [0, 0, 0, 0], [-0.5, 0, 0.5, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
M1.diattenuator_charac_angles_from_Jones(p1=1, p2=0, alpha=45*degrees, delay=180*degrees)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))


solution = np.matrix(
    np.array([[0.5, 0, 0, 0.5], [0, 0, 0, 0], [0, 0, 0, 0],
              [0.5, 0, 0, 0.5]]))
M1 = Mueller('M1')
M1.diattenuator_charac_angles_from_Jones(p1=1, p2=0, alpha=45*degrees, delay=90*degrees)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))


[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]
0.0
[[ 5.00000000e-01  1.11022302e-16 -5.00000000e-01  6.12323400e-17]
 [ 1.11022302e-16  0.00000000e+00 -1.11022302e-16  1.23259516e-32]
 [-5.00000000e-01 -1.11022302e-16  5.00000000e-01 -6.12323400e-17]
 [ 6.12323400e-17  1.23259516e-32 -6.12323400e-17  0.00000000e+00]]
2.535772158592562e-16
[[5.00000000e-01 1.11022302e-16 3.06161700e-17 5.00000000e-01]
 [1.11022302e-16 0.00000000e+00 6.16297582e-33 1.11022302e-16]
 [3.06161700e-17 6.16297582e-33 0.00000000e+00 3.06161700e-17]
 [5.00000000e-01 1.11022302e-16 3.06161700e-17 5.00000000e-01]]
2.303328157969851e-16


In [15]:
# diattenuator_azimuth_ellipticity_from_Jones
solution = np.matrix(
    np.array([[0.5, 0.5, 0, 0], [0.5, 0.5, 0, 0], [0, 0, 0, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
M1.diattenuator_azimuth_ellipticity_from_Jones(p1=1, p2=0, azimuth=0, ellipticity=0)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))


solution = np.matrix(
    np.array([[0.5, 0, -0.5, 0], [0, 0, 0, 0], [-0.5, 0, 0.5, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
M1.diattenuator_azimuth_ellipticity_from_Jones(p1=1, p2=0, azimuth=135*degrees, ellipticity=0)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))


solution = np.matrix(
    np.array([[0.5, 0, 0, 0.5], [0, 0, 0, 0], [0, 0, 0, 0],
              [0.5, 0, 0, 0.5]]))
M1 = Mueller('M1')
M1.diattenuator_azimuth_ellipticity_from_Jones(p1=1, p2=0, azimuth=45*degrees, ellipticity=45*degrees)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))


[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]
0.0
[[ 5.00000000e-01 -1.11022302e-16 -5.00000000e-01  6.12323400e-17]
 [-1.11022302e-16  0.00000000e+00  1.11022302e-16 -1.23259516e-32]
 [-5.00000000e-01  1.11022302e-16  5.00000000e-01 -6.12323400e-17]
 [ 6.12323400e-17 -1.23259516e-32 -6.12323400e-17  0.00000000e+00]]
2.535772158592562e-16
[[5.00000000e-01 1.11022302e-16 3.06161700e-17 5.00000000e-01]
 [1.11022302e-16 0.00000000e+00 6.16297582e-33 1.11022302e-16]
 [3.06161700e-17 6.16297582e-33 0.00000000e+00 3.06161700e-17]
 [5.00000000e-01 1.11022302e-16 3.06161700e-17 5.00000000e-01]]
2.303328157969851e-16


In [16]:
# diattenuator_charac_angles_from_vector
solution = np.matrix(
    np.array([[0.5, 0.5, 0, 0], [0.5, 0.5, 0, 0], [0, 0, 0, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
M1.diattenuator_charac_angles_from_vector(p1=1, p2=0, alpha=0, delay=0)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))


solution = np.matrix(
    np.array([[0.5, 0, -0.5, 0], [0, 0, 0, 0], [-0.5, 0, 0.5, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
M1.diattenuator_charac_angles_from_vector(p1=1, p2=0, alpha=45*degrees, delay=180*degrees)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))



solution = np.matrix(
    np.array([[0.5, 0, 0, 0.5], [0, 0, 0, 0], [0, 0, 0, 0],
              [0.5, 0, 0, 0.5]]))
M1 = Mueller('M1')
M1.diattenuator_charac_angles_from_vector(p1=1, p2=0, alpha=45*degrees, delay=90*degrees)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]
0.0
[[ 5.00000000e-01  3.06161700e-17 -5.00000000e-01  6.12323400e-17]
 [ 3.06161700e-17  1.87469973e-33 -3.06161700e-17  3.74939946e-33]
 [-5.00000000e-01 -3.06161700e-17  5.00000000e-01 -6.12323400e-17]
 [ 6.12323400e-17  3.74939946e-33 -6.12323400e-17  7.49879891e-33]]
1.3691967456605067e-16
[[5.00000000e-01 3.06161700e-17 3.06161700e-17 5.00000000e-01]
 [3.06161700e-17 1.87469973e-33 1.87469973e-33 3.06161700e-17]
 [3.06161700e-17 1.87469973e-33 1.87469973e-33 3.06161700e-17]
 [5.00000000e-01 3.06161700e-17 3.06161700e-17 5.00000000e-01]]
8.659560562354933e-17


In [17]:
# diattenuator_azimuth_ellipticity_from_vector
solution = np.matrix(
    np.array([[0.5, 0.5, 0, 0], [0.5, 0.5, 0, 0], [0, 0, 0, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
M1.diattenuator_azimuth_ellipticity_from_vector(p1=1, p2=0, azimuth=0, ellipticity=0)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))



solution = np.matrix(
    np.array([[0.5, 0, -0.5, 0], [0, 0, 0, 0], [-0.5, 0, 0.5, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
M1.diattenuator_azimuth_ellipticity_from_vector(p1=1, p2=0, azimuth=135*degrees, ellipticity=0)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))



solution = np.matrix(
    np.array([[0.5, 0, 0, 0.5], [0, 0, 0, 0], [0, 0, 0, 0],
              [0.5, 0, 0, 0.5]]))
M1 = Mueller('M1')
M1.diattenuator_azimuth_ellipticity_from_vector(p1=1, p2=0, azimuth=45*degrees, ellipticity=45*degrees)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]
0.0
[[ 5.00000000e-01 -8.04061325e-17 -5.00000000e-01  6.12323400e-17]
 [-8.04061325e-17  1.29302923e-32  8.04061325e-17 -9.84691128e-33]
 [-5.00000000e-01  8.04061325e-17  5.00000000e-01 -6.12323400e-17]
 [ 6.12323400e-17 -9.84691128e-33 -6.12323400e-17  7.49879891e-33]]
2.021340703361228e-16
[[5.00000000e-01 3.06161700e-17 3.06161700e-17 5.00000000e-01]
 [3.06161700e-17 1.87469973e-33 1.87469973e-33 3.06161700e-17]
 [3.06161700e-17 1.87469973e-33 1.87469973e-33 3.06161700e-17]
 [5.00000000e-01 3.06161700e-17 3.06161700e-17 5.00000000e-01]]
8.659560562354933e-17


In [18]:
# diattenuator_from_vector
solution = np.matrix(
    np.array([[0.5, 0.5, 0, 0], [0.5, 0.5, 0, 0], [0, 0, 0, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
D = np.matrix(np.array([1, 0, 0]))
m00 = 0.5
M1.diattenuator_from_vector(D, m00)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))



solution = np.matrix(
    np.array([[0.5, 0, -0.5, 0], [0, 0, 0, 0], [-0.5, 0, 0.5, 0],
              [0, 0, 0, 0]]))
M1 = Mueller('M1')
D = np.matrix(np.array([0, -1, 0]))
M1.diattenuator_from_vector(D, m00)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))



solution = np.matrix(
    np.array([[0.5, 0, 0, 0.5], [0, 0, 0, 0], [0, 0, 0, 0],
              [0.5, 0, 0, 0.5]]))
M1 = Mueller('M1')
D = np.matrix(np.array([0, 0, 1]))
m00 = 0.5
M1.diattenuator_from_vector(D, m00)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]
0.0
[[ 0.5  0.  -0.5  0. ]
 [ 0.   0.   0.   0. ]
 [-0.5  0.   0.5  0. ]
 [ 0.   0.   0.   0. ]]
0.0
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]
0.0


**Retarders**

In [19]:
# retarder_charac_angles_from_Jones
solution = np.matrix(
    np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0],
              [0, 0, 0, -1]]))
M1 = Mueller('M1')
M1.retarder_charac_angles_from_Jones(D=180*degrees, alpha=0, delta=0, m00=1)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

r2 = sqrt(2)/2
solution = np.matrix(
    np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, r2, r2],
              [0, 0, -r2, r2]]))
M1 = Mueller('M1')
M1.retarder_charac_angles_from_Jones(D=45*degrees, alpha=0, delta=0, m00=1)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

r2 = sqrt(2)/2
solution = np.matrix(
    np.array([[1, 0, 0, 0], [0, r2, r2, 0], [0, -r2, r2, 0],
              [0, 0, 0, 1]]))
M1 = Mueller('M1')
M1.retarder_charac_angles_from_Jones(D=45*degrees, alpha=45*degrees, delta=90*degrees, m00=1)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  1.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00 -1.0000000e+00  1.2246468e-16]
 [ 0.0000000e+00  0.0000000e+00 -1.2246468e-16 -1.0000000e+00]]
1.7319121124709866e-16
[[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          0.70710678  0.70710678]
 [ 0.          0.         -0.70710678  0.70710678]]
1.5700924586837752e-16
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  7.07106781e-01  7.07106781e-01  4.16749887e-17]
 [ 0.00000000e+00 -7.07106781e-01  7.07106781e-01  2.23077003e-16]
 [ 0.00000000e+00  1.28270594e-16 -1.87207929e-16  1.00000000e+00]]
3.5728449608497014e-16


In [4]:
# retarder_azimuth_ellipticity_from_Jones
solution = np.matrix(
    np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0],
              [0, 0, 0, -1]]))
M1 = Mueller('M1')
M1.retarder_azimuth_ellipticity_from_Jones(D=180*degrees, azimuth=0, ellipticity=0, m00=1)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

r2 = sqrt(2)/2
solution = np.matrix(
    np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, r2, r2],
              [0, 0, -r2, r2]]))
M1 = Mueller('M1')
M1.retarder_azimuth_ellipticity_from_Jones(D=45*degrees, azimuth=0*degrees, ellipticity=0, m00=1)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

r2 = sqrt(2)/2
solution = np.matrix(
    np.array([[1, 0, 0, 0], [0, r2, r2, 0], [0, -r2, r2, 0],
              [0, 0, 0, 1]]))
M1 = Mueller('M1')
M1.retarder_azimuth_ellipticity_from_Jones(D=45*degrees, azimuth=0, ellipticity=45*degrees, m00=1)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  1.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00 -1.0000000e+00  1.2246468e-16]
 [ 0.0000000e+00  0.0000000e+00 -1.2246468e-16 -1.0000000e+00]]
1.7319121124709866e-16
[[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          0.70710678  0.70710678]
 [ 0.          0.         -0.70710678  0.70710678]]
1.5700924586837752e-16
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  7.07106781e-01  7.07106781e-01 -4.32978028e-17]
 [ 0.00000000e+00 -7.07106781e-01  7.07106781e-01  1.79345371e-17]
 [ 0.00000000e+00  4.32978028e-17  1.79345371e-17  1.00000000e+00]]
8.033319240475757e-16


In [3]:
# retarder_charac_angles_from_vector
r2 = sqrt(2)/2
solution = np.matrix(
    np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, r2, r2],
              [0, 0, -r2, r2]]))
M1 = Mueller('M1')
M1.retarder_charac_angles_from_vector(D=45*degrees, alpha=0, delta=0, m00=1)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

r2 = sqrt(2)/2
solution = 0.8*np.matrix(
    np.array([[1, 0, 0, 0], [0, r2, r2, 0], [0, -r2, r2, 0],
              [0, 0, 0, 1]]))
M1 = Mueller('M1')
M1.retarder_charac_angles_from_vector(D=45*degrees, alpha=45*degrees, delta=90*degrees, m00=0.8)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          0.70710678  0.70710678]
 [ 0.          0.         -0.70710678  0.70710678]]
0.0
[[ 8.00000000e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  5.65685425e-01  5.65685425e-01 -2.02906125e-17]
 [ 0.00000000e+00 -5.65685425e-01  5.65685425e-01  4.89858720e-17]
 [ 0.00000000e+00  4.89858720e-17 -2.02906125e-17  8.00000000e-01]]
7.498432648522078e-17


In [5]:
# retarder_azimuth_ellipticity_from_vector
r2 = sqrt(2)/2
solution = np.matrix(
    np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, r2, r2],
              [0, 0, -r2, r2]]))
M1 = Mueller('M1')
M1.retarder_azimuth_ellipticity_from_vector(D=45*degrees, azimuth=0*degrees, ellipticity=0, m00=1)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

r2 = sqrt(2)/2
solution = 0.8*np.matrix(
    np.array([[1, 0, 0, 0], [0, r2, r2, 0], [0, -r2, r2, 0],
              [0, 0, 0, 1]]))
M1 = Mueller('M1')
M1.retarder_azimuth_ellipticity_from_vector(D=45*degrees, azimuth=0, ellipticity=45*degrees, m00=0.8)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          0.70710678  0.70710678]
 [ 0.          0.         -0.70710678  0.70710678]]
0.0
[[ 8.00000000e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  5.65685425e-01  5.65685425e-01 -2.02906125e-17]
 [ 0.00000000e+00 -5.65685425e-01  5.65685425e-01  4.89858720e-17]
 [ 0.00000000e+00  4.89858720e-17 -2.02906125e-17  8.00000000e-01]]
7.498432648522078e-17


In [6]:
# retarder_from_vector
solution = np.matrix(
    np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0],
              [0, 0, 0, -1]]))
try:
    M1 = Mueller('M1')
    M1.retarder_from_vector(D=180*degrees, ur=None, m00=1)
    proposal = M1.M
    print(proposal)
    print(np.linalg.norm(proposal - solution))
except:
    print(0.0)


r2 = sqrt(2)/2
solution = np.matrix(
    np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, r2, r2],
              [0, 0, -r2, r2]]))
M1 = Mueller('M1')
M1.retarder_from_vector(D=45*degrees, ur = np.array([1, 0, 0]), m00=1)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

r2 = sqrt(2)/2
solution = 0.8*np.matrix(
    np.array([[1, 0, 0, 0], [0, r2, r2, 0], [0, -r2, r2, 0],
              [0, 0, 0, 1]]))
M1 = Mueller('M1')
M1.retarder_from_vector(D=45*degrees, ur = np.array([0, 0, 1]), m00=0.8)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

0.0
[[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          0.70710678  0.70710678]
 [ 0.          0.         -0.70710678  0.70710678]]
0.0
[[ 0.8         0.          0.          0.        ]
 [ 0.          0.56568542  0.56568542  0.        ]
 [ 0.         -0.56568542  0.56568542  0.        ]
 [ 0.          0.          0.          0.8       ]]
0.0


In [4]:
# diattenuator_retarder_linear
r3p2 = sqrt(3)/2
solution = 0.5 * np.matrix(
    np.array([[1.25, 0.75, 0, 0], [0.75, 1.25, 0, 0], [0, 0, 0.5, r3p2],
              [0, 0, -r3p2, 0.5]]))
M1 = Mueller('M1')
M1.diattenuator_retarder_linear(p1=1, p2=sqrt(0.25), D=60*degrees, angle=0)
proposal = M1.M

print(proposal)
print(np.linalg.norm(proposal - solution))

[[ 0.625      0.375      0.         0.       ]
 [ 0.375      0.625      0.         0.       ]
 [ 0.         0.         0.25       0.4330127]
 [ 0.         0.        -0.4330127  0.25     ]]
7.850462293418876e-17


In [5]:
a = np.zeros([3,3])
print(a)
a[0,0] = Mueller()
print(a)

[autoreload of py_pol.mueller failed: Traceback (most recent call last):
  File "C:\Users\Jesus\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Jesus\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 376, in superreload
    module = reload(module)
  File "C:\Users\Jesus\Anaconda3\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\Users\Jesus\Anaconda3\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "d:\codigo\py_pol\py_pol\mueller.py", line 563
    se

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


TypeError: float() argument must be a string or a number, not 'Mueller'

In [8]:
print(isinstance("hola", str))

True


In [11]:
a = [0, 1, 2, 3, 4 ,5, 6]
N = len(a)
print(a[1:N+1])

[1, 2, 3, 4, 5, 6]


In [3]:
print(list_of_objects(3,Mueller))

[  = 
  [ +0,  +0,  +0,  +0]
  [ +0,  +0,  +0,  +0]
  [ +0,  +0,  +0,  +0]
  [ +0,  +0,  +0,  +0]
,   = 
  [ +0,  +0,  +0,  +0]
  [ +0,  +0,  +0,  +0]
  [ +0,  +0,  +0,  +0]
  [ +0,  +0,  +0,  +0]
,   = 
  [ +0,  +0,  +0,  +0]
  [ +0,  +0,  +0,  +0]
  [ +0,  +0,  +0,  +0]
  [ +0,  +0,  +0,  +0]
]
